## GraphQL query with Obsidian Notes

I am exposing the notes as a local GraphQL server using this [plugin](https://github.com/TwIStOy/obsidian-local-graphql). <br>
Access the Apollo client UI by going to this link [http://localhost:28123/](http://localhost:28123/)

### Notes:
- The score returned indicating the match relevancy to the search query is arbitrary, there is no description of how this is derived from the obsidian docs. 
- I set a minimum negative score for results to be displayed, the closer to 0 the better.
- I have also added a `top_n` parameter, as many results may be under the threshold for negative score. This is to ensure we are not passing massive amounts of context to the LLM
- The simple search really only works when words are spelled exactly as they can be found within the documents - Probably should only be used when a list of the titles or specific tags are give to the LLM for context, so they can do a top down search by category. *For example: search "generative AI" and get all the child documents to that topic.*
- The fuzzy search works well for somewhat similar spelling. Good for specific searches that may not be broad topics or categories.
- Queries should use search terms like "QLoRA", and not questions like "What is QLoRA?" as the search functionality will look for "What is..." which may not be in the documents.

In [ ]:
# ! curl --request POST \
#   --header 'content-type: application/json' \
#   --url http://localhost:28123/ \
#   --data '{"query":"query { __typename }"}'

In [1]:
import json
import networkx as nx
from pyvis.network import Network

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:

# Configure the client
transport = RequestsHTTPTransport(
    url='http://localhost:28123/',  # Make sure this URL is correct
    use_json=True,
    headers={'Content-Type': 'application/json'},
    verify=True,
    retries=3,
)

client = Client(transport=transport, fetch_schema_from_transport=True)


def define_query_string(search_type):
    # Define your query with the variable
    string = f""" \
      query ExampleQuery($query: String!) {{
        vault {{
          {search_type}(query: $query) {{
            file {{
              name
              readContent
              stat {{
                size
              }}
            }}
            score
          }}
        }}
      }}
    """
    return gql(string)

In [12]:

def plot_graph(question, result, search_type, min_neg_score=-3):
    # Create a graph
    G = nx.Graph()
    G.add_node(question, size=20, title=question)
    # Parse JSON and add nodes and edges
    for item in result["vault"][search_type]:
        if item["score"] > min_neg_score:
            file_name = item["file"]["name"]
            score = item["score"]

            # Adding each file as a node
            G.add_node(file_name, size=10, title=f"Score: {score}")
            
            # Adding an edge from each file to the central query node, weight is the inverse of score for visualization
            G.add_edge(question, file_name, weight=score)

    # Convert to a PyVis network for visualization
    nt = Network("500px", "1000px", notebook=True)
    nt.from_nx(G)
    nt.show(f"fuzzy_search_graph_{search_type}.html")

def print_result(result, min_neg_score, search_type, top_n):
    # Filter results based on min_neg_score and sort by score descending (closest to 0)
    top_items = sorted(
        (item for item in result["vault"][search_type] if item["score"] > min_neg_score),
        key=lambda x: x["score"], 
        reverse=True
    )

    # Get top N results
    if top_n != -1:
        top_items = top_items[:top_n]

    for item in top_items:
        file_name = item["file"]["name"]
        score = item["score"]
        read_content = item['file']['readContent']

        print(f"File Name: {file_name}")
        print(f"Score: {score}")
        print(f"Content:\n{read_content}")
        print(">>>>-----------------------------")



In [4]:
SIMPLE_SEARCH_TYPE = "simpleSearch"
FUZZY_SEARCH_TYPE = "fuzzySearch"
MIN_NEG_SCORE = -3
TOP_N = 3

simple_search_query = define_query_string(SIMPLE_SEARCH_TYPE)
fuzzy_search_query = define_query_string(FUZZY_SEARCH_TYPE)

## Simple Search vs Fuzzy Search

Fuzzy search is a feature that returns results based on likely relevance even if the search terms are not an exact match. This means that even if the words or spellings do not precisely match, the search results will still be displayed based on relevance. Fuzzy search is useful when searching for terms where variations or misspellings might occur, ensuring that relevant results are still provided. It helps save time by offering suggestions and matches that are closely related to the search query, even if not identical.

In [14]:
# Execute the query with variables
variables = {
    "query": "LoRA"
}

result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

File Name: Quantization.md
Score: -0.08
Content:
> [[Large Language Model]], [[QLoRA]],
### What is Quantization?

![[Screenshot 2024-03-05 at 13.40.43.png]]

- Quantizing involves splitting a range of numbers (which is infinite in floating point) into bins
- Quantization is needed when you want to represent a value from an infinite range in a physically constrained system, as you need infinite bytes - you need to make some assumptions
>>>>-----------------------------
File Name: Pearl.md
Score: -0.5349
Content:

Pearl is a production-ready Reinforcement Learning (RL) AI agent library developed by Meta's Applied Reinforcement Learning team. It's designed to enable researchers and practitioners to develop RL AI agents that prioritize long-term over immediate feedback and adapt to environments with limited observability, sparse feedback, and high stochasticity. Key features include modular design, dynamic action spaces, offline learning, intelligent neural exploration, safe decision maki

In [13]:
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

File Name: Quantization.md
Score: -0.18000000000000002
Content:
> [[Large Language Model]], [[QLoRA]],
### What is Quantization?

![[Screenshot 2024-03-05 at 13.40.43.png]]

- Quantizing involves splitting a range of numbers (which is infinite in floating point) into bins
- Quantization is needed when you want to represent a value from an infinite range in a physically constrained system, as you need infinite bytes - you need to make some assumptions
>>>>-----------------------------
File Name: LoRA.md
Score: -0.25470000000000004
Content:
---
tags:
  - fine-tuning
  - llm
  - generative-ai
---
> [[Large Language Model]], [[Generative AI]], [[Fine Tuning]]

LoRA Fine-tuning refers to a technique for improving the performance of a pre-trained language model by fine-tuning it on a specific task or domain. LoRA stands for "Language-oriented Recurrent Attention", which is a neural architecture that has been used for pre-training language models.

Fine-tuning a pre-trained language model inv

In [ ]:
# Assuming plot_graph is a function you have defined earlier to create and display the graph
# plot_graph(variables["query"], result, FUZZY_SEARCH_TYPE)

---

In [15]:
variables = {
    "query": "QLoRA"
}

result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

File Name: Quantization.md
Score: -0.079
Content:
> [[Large Language Model]], [[QLoRA]],
### What is Quantization?

![[Screenshot 2024-03-05 at 13.40.43.png]]

- Quantizing involves splitting a range of numbers (which is infinite in floating point) into bins
- Quantization is needed when you want to represent a value from an infinite range in a physically constrained system, as you need infinite bytes - you need to make some assumptions
>>>>-----------------------------
File Name: QLoRA.md
Score: -1.5467
Content:
---
tags:
  - fine-tuning
  - generative-ai
---
> [[Fine Tuning]], [[Fine Tuning]], [[Generative AI]], [[Quantization]]
### QLoRA

There are 4 components to QLoRA:
1. **4-bit NormalFloat**
	- This is essentially just a way to bucket values
	- Representing a value within 4 binary values
	- *ex: `0101`*
	- 16 Unique unique combinations ![[Screenshot 2024-03-05 at 13.48.24.png]]
2. **Double Quantization**
	- 
3. **Paged Optimizers**
	- Used to provide higher utilization of memory

In [16]:
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

File Name: Quantization.md
Score: -0.079
Content:
> [[Large Language Model]], [[QLoRA]],
### What is Quantization?

![[Screenshot 2024-03-05 at 13.40.43.png]]

- Quantizing involves splitting a range of numbers (which is infinite in floating point) into bins
- Quantization is needed when you want to represent a value from an infinite range in a physically constrained system, as you need infinite bytes - you need to make some assumptions
>>>>-----------------------------
File Name: QLoRA.md
Score: -0.2167
Content:
---
tags:
  - fine-tuning
  - generative-ai
---
> [[Fine Tuning]], [[Fine Tuning]], [[Generative AI]], [[Quantization]]
### QLoRA

There are 4 components to QLoRA:
1. **4-bit NormalFloat**
	- This is essentially just a way to bucket values
	- Representing a value within 4 binary values
	- *ex: `0101`*
	- 16 Unique unique combinations ![[Screenshot 2024-03-05 at 13.48.24.png]]
2. **Double Quantization**
	- 
3. **Paged Optimizers**
	- Used to provide higher utilization of memory

---

In [25]:
variables = {
    "query": "What is QLoRA?"
}

# Execute the query with variables
result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

In [31]:
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

---

In [40]:
variables = {
    "query": "Machine Learning Operations"
}

# Execute the query with variables
result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

File Name: 24 - Choosing the Best Performing Model.md
Score: -2.1782
Content:
---
tags:
  - ml
  - mlops
  - ml-pipelines
---
> [[Machine Learning]], [[ML Operations]], [[ML Pipelines]], [[Model Evaluation]]
### Choose the Best Performing Model
- Go to table visualisation for a group of experiment runs
- There are many columns, you can customise
- Sort by loss/accuracy
- You can get the parameters for the best run

![[download-2 2.png]]
>>>>-----------------------------
File Name: 4. Error analysis and Performance Auditing.md
Score: -2.1866999999999996
Content:
---
tags:
  - mlops
  - ml
---
> [[ML Operations]],  [[Machine Learning]]
> 
Using the covariates (tags) in the test data set can be useful for error analysis. You can also use supplementary meta-data to determine implicit root causes of errors

Questions to ask:
- What % of errors has a given tag?
- Of all data with that tag, what fraction is misclassified?
- What fraction of data has that tag?

![[download-4 1.png]]
## Perform

In [43]:
# Execute the query with variables
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

File Name: 24 - Choosing the Best Performing Model.md
Score: -2.1782
Content:
---
tags:
  - ml
  - mlops
  - ml-pipelines
---
> [[Machine Learning]], [[ML Operations]], [[ML Pipelines]], [[Model Evaluation]]
### Choose the Best Performing Model
- Go to table visualisation for a group of experiment runs
- There are many columns, you can customise
- Sort by loss/accuracy
- You can get the parameters for the best run

![[download-2 2.png]]
>>>>-----------------------------
File Name: 5 - Model Cards.md
Score: -2.1990999999999996
Content:
---
tags:
  - ml
  - mlops
  - ml-pipelines
  - deployment
---
> > [[Machine Learning]], [[ML Operations]], [[ML Pipelines]], [[Deployment]], 
### Model Cards
- Encapsulate all the key details about the model for future reference
- It’s like a memo for your model
- Should be understandable
- Should display how the model:
	- Data used
	- How to use it
	- And its shortcomings
	- Metrics
	- Parameters
>>>>-----------------------------
File Name: 4. Versionin

---

In [44]:
variables = {
    "query": "PU Learning"
}

# Execute the query with variables
result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

File Name: Reinforcement Learning from Human Feedback (RLHF).md
Score: -2.2328
Content:
---
tags:
  - fine-tuning
  - llm
  - generative-ai
---
 > [[Large Language Model]], [[Generative AI]], [[Fine Tuning]]

**Reinforcement Learning from Human Feedback (RLHF):**
   - RLHF is an advanced technique wherein human reviewers rate model outputs to guide its fine-tuning.
   - It's a complex process, often reserved for organizations with substantial resources.
   - A prominent example of RLHF is ChatGPT by OpenAI, which underwent several stages of fine-tuning based on human feedback.
[OpenAI's InstructGPT Paper](https://www.openai.com/research/)
>>>>-----------------------------
File Name: Untitled.md
Score: -2.3185
Content:

Learning embedded systems often involves understanding both software and hardware. It requires knowledge of electrical engineering, computer architecture, programming (typically in C or C++), and sometimes real-time operating systems. The challenge in embedded systems li

In [45]:
# Execute the query with variables
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

File Name: Positive-Unlabelled (PU) Learning.md
Score: -1.1520000000000001
Content:
---
tags:
  - ml
---
> [[Machine Learning]]
### What is PU Learning?
PU learning, which stands for positive and unlabelled learning, is a semi-supervised binary classification method that recovers labels from unknown cases in the data. It does this by learning from the positive cases in the data and applying what it has learned to relabel the unknown cases. This approach provides benefits to any machine learning problem that requires binary classification on unreliable data, regardless of the domain.

There are two main approaches to applying PU learning. These include:
- PU bagging 
- Two-step approach

[https://heartbeat.fritz.ai/positive-and-unlabelled-learning-recovering-labels-for-data-using-machine-learning-59c1def5452f](https://heartbeat.fritz.ai/positive-and-unlabelled-learning-recovering-labels-for-data-using-machine-learning-59c1def5452f)
>>>>-----------------------------
File Name: Apple Neur

---

In [46]:
variables = {
    "query": "Model Bias"
}

# Execute the query with variables
result = client.execute(simple_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, SIMPLE_SEARCH_TYPE, TOP_N)

In [47]:
# Execute the query with variables
result = client.execute(fuzzy_search_query, variable_values=variables)
print_result(result, MIN_NEG_SCORE, FUZZY_SEARCH_TYPE, TOP_N)

File Name: 8. Aequitas.md
Score: -1.1400000000000001
Content:
---
tags:
  - data
  - ml
  - model-bias
  - mlops
  - model-testing
---
> [[Model Evaluation]], [[ML Pipelines]], [[Model Bias (Fairness)]], [[7.  Model Bias]]
### Aequitas

Aequitas is an open-source bias audit toolkit that allows users to evaluate their machine learning models for fairness. It can be used both as a Python library or via a web interface. Aequitas offers various fairness metrics and visualizations to help you understand and address bias in your model. Here are the requirements for using Aequitas:
- Model score
- Label
- Binary classification only
- At least one categorical feature OR a numerical feature

There are 3 reports generated:
- Create cross-tabulation using Group() class(this is the basis for all analysis)
- After the cross tab we compute bias using Bias() class
- Then we compute fairness using Fairness() class
- This can result in a beefy data frame 



[http://aequitas.dssg.io/](http://aequitas.d